<a href="https://colab.research.google.com/github/SilverSurferClash/Regression_templates/blob/main/feb_2021_nn_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Mounted at /content/drive


In [5]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 924 kB/s 


In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
#Import os libaries
import os
import time
from pathlib import Path
import sys
import pickle
import joblib

#Import data manipulation libaries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#import ploting libraries
import matplotlib.pyplot as plt
import seaborn as sns

#Splitting the data and cross-validation
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, cross_val_score,KFold, GroupKFold, StratifiedKFold

#Metrics
#from sklearn.metrics import classification_report,accuracy_score , roc_auc_score
from sklearn.metrics import mean_squared_error
#Classifiers
import xgboost as xgb
from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

#Feature engineerring
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer, make_column_transformer

#Encoding

import category_encoders as ce

#Helper modules
from tqdm import tqdm

In [8]:
#Notebook settings
# To define maximum number of columns to be displayed in a dataframe
pd.set_option('display.max_columns', None)

# To supress warnings
import warnings
warnings.filterwarnings('ignore')

#Set the theme for seaborn
#sns.set_theme(style="darkgrid")
pd.set_option('display.precision', 2)

#Set dark theme
plt.style.use('dark_background')

In [9]:
os.getcwd()

'/content'

In [10]:
os.chdir("/content/drive/MyDrive/Colab_Notebooks/deep_learning")

In [11]:
train_import = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

#### List of different feature sets

In [12]:
# List of colums based on their function
# Target
target_label  = "target"
# List of all features
all_feature = [col for col in train_import.columns if col != "target"]
# List of numeric features
all_numeric = [col for col in all_feature if train_import[col].dtype == "float64"]
# List of all categorical features
all_categorical = [col for col in all_feature if train_import[col].dtype == "object"]

In [13]:
#Step 0: Drop non-essential columns
#Step 1: Perform imputation or encoding of missing data
#Step 2: Perform encoding for the different categorical columns
#Step 3: feature transformation of numeric features
#Step 4: feature generation by aggregation/groupby function
#Step 5: 

### Create the feature matrix and target values

In [14]:
#Create the training feature matrix, the training feature labels and the same for the
#validation set

train = train_import.sample(frac = 0.05).copy()
train_X = train.loc[:, all_feature]
train_y = train.loc[:, [target_label]].values
train_X.shape, train_y.shape

((15000, 25), (15000, 1))

### Feature Engineering - Encoding and Scaling

In [15]:
preprocess_continuous = Pipeline(steps=[('scaler', StandardScaler())])
preprocess_categorical = Pipeline(steps=[('encoder',
ce.OneHotEncoder(cols=all_categorical))])

In [16]:
ct = ColumnTransformer(
transformers=[('continuous', #A
preprocess_continuous, all_numeric),
('categorical', #B
preprocess_categorical, all_categorical)],
remainder='passthrough')

In [17]:
train_X_trans = ct.fit_transform(train_X, train_y)
test_trans = ct.transform(test)

In [18]:
train_data = train_X_trans.copy()
train_targets = train_y.copy()


In [25]:
type(train_data), type(train_targets), train_targets.shape, train_data.shape

(numpy.ndarray, numpy.ndarray, (15000, 1), (15000, 68))

Create Neural network

In [26]:
from tensorflow import keras
from tensorflow.keras import layers

In [36]:
# Initialize the model
# define the model

def build_model():
  model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(1)
    ])



  model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
  return model

In [32]:
#Compile the model - we need a:
#loss function
#optimizer
# evaluation metric

#model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

In [35]:
# Define the number of epochs for training
num_epochs = 5

In [37]:
#build_model().fit(train_data,train_targets,
             # epochs=num_epochs, batch_size=16, verbose=1)

Epoch 1/5
938/938 [==============================] - 2s 2ms/step - loss: 4421246.0000 - mae: 956.1060
Epoch 2/5
938/938 [==============================] - 1s 2ms/step - loss: 875022.1875 - mae: 500.8616
Epoch 3/5
938/938 [==============================] - 2s 2ms/step - loss: 211172.4688 - mae: 277.4656
Epoch 4/5
938/938 [==============================] - 1s 2ms/step - loss: 21458.2578 - mae: 62.8584
Epoch 5/5
938/938 [==============================] - 1s 2ms/step - loss: 100.3504 - mae: 3.6847


In [42]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

In [62]:
for i in range(k):
  print('processing fold #', i)
  # Take the i fold for the validation data - shift everything by i+1
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data = np.concatenate(
  [train_data[:i * num_val_samples],
  train_data[(i + 1) * num_val_samples:]],
  axis=0)
  partial_train_targets = np.concatenate(
  [train_targets[:i * num_val_samples],
  train_targets[(i + 1) * num_val_samples:]],
  axis=0)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
